In [ ]:
import logging
import asyncio
import pandas as pd
import polars as pl

from datetime import datetime, timedelta

import influxdb_client

from tastytrade.common.logging import setup_logging
from tastytrade.config.enumerations import Channels
from tastytrade.connections.sockets import DXLinkManager
from tastytrade.connections import Credentials, InfluxCredentials

from tastytrade.messaging.processors.influxdb import TelegrafHTTPEventProcessor

# Show all rows in pandas DataFrames
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

logging.getLogger().handlers.clear()

TEST = True
ENV = "Live"
DURATION = 15

EDT = 5

start_time = datetime(2025, 1, 1)

setup_logging(
    level=logging.INFO,
    log_dir="../logs",
    filename_prefix=f"{'dev' if TEST else 'prod'}_tastytrade",
    console=True,
    file=True,
)

loop = asyncio.get_event_loop()
loop.set_debug(True)
logging.getLogger("asyncio").setLevel(logging.DEBUG)

## Test individual components

In [ ]:
credentials = Credentials(env="Live")
dxlink = DXLinkManager()

await dxlink.open(credentials=credentials)

In [3]:
influx_user = InfluxCredentials()
influxdb = influxdb_client.InfluxDBClient(  # **InfluxCredentials().__dict__)
    url=influx_user.url, token=influx_user.token, org=influx_user.org
)
query_api = influxdb.query_api()

In [4]:
for handler in dxlink.router.handler.values():
    handler.add_processor(TelegrafHTTPEventProcessor())

In [ ]:
dxlink.router.handler[Channels.Candle].processors

In [ ]:
start_time = datetime(2025, 2, 25, 9)

# for symbol in ["NVDA"]:
# for symbol in ["BTC/USD:CXTALP", "NVDA", "QQQ", "SPY", "SPX"]:

for symbol in ["BTC/USD:CXTALP", "NVDA", "QQQ", "SPY", "SPX"]:
    for interval in ["1d", "1h", "30m", "15m", "5m", "m"]:
        await dxlink.subscribe_to_candles(
            symbol=symbol,
            interval=interval,
            from_time=start_time,
        )

In [ ]:
symbols = ["NVDA"]
symbols = ["SPX"]
symbols = ["SPX", "NVDA", "SPY", "QQQ"]
symbols = ["BTC/USD:CXTALP", "SPX", "NVDA", "SPY", "QQQ"]
await dxlink.subscribe(symbols)

In [ ]:
symbol = "SPX{=m}"
symbol = "BTC/USD:CXTALP{=m}"

dxlink.router.handler[Channels.Candle].processors["feed"].frames[f"{symbol}"].tail(1)

In [ ]:
dxlink.router.handler[Channels.Candle].processors["feed"].frames[f"{symbol}"].shape

In [9]:
# TODOS

# [x] Widen the plot
# [x] Remove the scroller at the bottom
# [x] move the legent and remove Price (that is obvious)
# [x] Add MACD
# [x] Fix HULL - Align w/ candlesticks
# [x] ERROR if no study data found

# [ ] Add RSI
# [ ] Add Volume Profile (?? ... /ES, SPY, etc)
# [ ] Add velocity metric

In [8]:
from tastytrade.analytics.visualizations.plots import plot_macd_with_hull
from tastytrade.analytics.indicators.momentum import macd
from tastytrade.messaging.models.events import CandleEvent
import re

from tastytrade.providers.market import MarketDataProvider

streamer = MarketDataProvider(dxlink, influxdb)

# TODO Throw an error if stop <= start
start = datetime(2025, 2, 25, 9) + timedelta(hours=5)
stop = datetime(2025, 2, 25, 16) + timedelta(hours=5)
# stop = None

In [9]:
# import importlib
# import tastytrade.analytics.visualizations.plots

# importlib.reload(tastytrade.analytics.visualizations.plots)
# plot_macd_with_hull = tastytrade.analytics.visualizations.plots.plot_macd_with_hull

In [10]:
candle_symbol = "SPX{=m}"

prior_day: CandleEvent = CandleEvent(
    **(
        streamer.download(
            symbol=re.sub(r"\{=.*?\}", "{=d}", candle_symbol),
            start=start.date() + timedelta(days=-1),
            stop=start.date(),
            debug_mode=True,
        )
        .to_dicts()
        .pop()
    )
)

In [11]:
candles: pl.DataFrame = streamer.download(
    symbol=candle_symbol,
    start=start,
    stop=stop,
    debug_mode=True,
)

In [12]:
df_macd = macd(candles, prior_close=prior_day.close, fast_length=12, slow_length=26, macd_length=9)

In [ ]:
plot_macd_with_hull(df_macd, pad_value=prior_day.close)

In [14]:
candle_symbol = "SPX{=5m}"

candles: pl.DataFrame = streamer.download(
    symbol=candle_symbol,
    start=start,
    stop=stop,
    debug_mode=True,
)

In [ ]:
df_macd = macd(candles, prior_close=prior_day.close, fast_length=12, slow_length=26, macd_length=9)
plot_macd_with_hull(df_macd, pad_value=prior_day.close)

In [ ]:
candles.sort("time", descending=True)

In [ ]:
await dxlink.close()